In [1]:
# %pip install transformers
# %pip install --upgrade --force-reinstall scipy

In [ ]:

import transformers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ast
import os
import re

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split


from sklearn import metrics
from tqdm import tqdm
from torch.utils.data import random_split
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score



/work/hpc/miniconda3/envs/hieu-env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/work/hpc/miniconda3/envs/hieu-env/lib/python3.12/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
import torch

num_gpus = torch.cuda.device_count()
gpu_list = [torch.cuda.get_device_name(i) for i in range(num_gpus)]

print("Available GPUs:")
for i, gpu in enumerate(gpu_list):
    print(f"GPU {i}: {gpu}")


Available GPUs:
GPU 0: Tesla V100-SXM2-16GB
GPU 1: Tesla V100-SXM2-16GB
GPU 2: Tesla V100-SXM2-16GB
GPU 3: Tesla V100-SXM2-16GB


In [4]:
device = 'cuda:0'

In [5]:
args = {
    "model_name": "xlm_roberta",
    "max_length": 300,
    "batch_size": 8,
    "tranfer_learning_rate": 1e-3,
    "transfer_epochs": 1,
    "finetune_learning_rate": 1e-5,
    "fine_tune_epochs": 10,
    "device": device,
    "seed": 42,
    "test_size": 0.2,
    "train_size": 0.8,
}

In [7]:
# import the library
import wandb
wandb.login(key = 'a8d3f2f19fe58ead28cafb05be2d1cc8931944b3')

wandb: Currently logged in as: hieuvut123. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/hpc/.netrc


True

In [9]:
# start a new experiment
wandb.init(project="xlm_roberta_origin_data")
# capture a dictionary of hyperparameters with config
wandb.config = {"learning_rate": args['finetune_learning_rate'], "epochs": args['fine_tune_epochs'], "batch_size": args['batch_size']}



# **Importing and Pre-Processing the domain data**

In [10]:
train_path = 'origin/train.csv'
val_path = 'origin/val.csv'
test_path = 'origin/test.csv'

In [11]:
train_data = pd.read_csv(train_path)
val_data = pd.read_csv(val_path)    
test_data = pd.read_csv(test_path)

ParserError: Error tokenizing data. C error: EOF inside string starting at row 6344

In [11]:
train_data['text'] = train_data['text'].apply(str)
val_data['text'] = val_data['text'].apply(str)
test_data['text'] = test_data['text'].apply(str)

In [12]:
train_data

,id,text,label,dataset_type
0,1,Chắc cũng biết ko tồn tại đc bao lâu nữa nên c...,1,train
1,9,Emma Nguyen vậy mà ca ngợi bên kia lắm... Về l...,1,train
2,11,Nhung Pham Ai chê nghèo ngta có tiền đi nước n...,1,train
3,13,Ủa bê đê có lol hả ta ??😂😂😂,1,train
4,16,Làm thêm chục củ mở hòm csgo anh êi :V,0,train
...,...,...,...,...
22999,46002,"trong trường hợp này, theo ls. khanh, bà hằng...",1,test
23000,46003,bà cũng không ít lần chỉ trích nặng nề những ...,0,test
23001,46004,“bà nguyễn phương hằng có nhiều biểu hiện đã ...,0,test
23002,46005,vũ đức khanh nói. bà nguyễn phương hằng bắt đ...,0,test


In [13]:
train_data.label.value_counts()

label
0    19039
1     3965
Name: count, dtype: int64

In [14]:
def up_sample(df: pd.DataFrame):
    df_0 = df[df.label==0]
    df_1 = df[df.label==1]
    df_1_upsampled = df_1.sample(n=len(df_0), replace=True)
    return pd.concat([df_0, df_1_upsampled]).sample(frac=1)

In [15]:
def split_df(df: pd.DataFrame):
    '''
    test size: 0 -> 1
    '''
    
    return train_test_split(df, test_size= args['test_size'], random_state=42)

In [16]:
def process_train(df: pd.DataFrame) -> pd.DataFrame:
    df = df.dropna()
    df = df.drop_duplicates()
    df = df.reset_index(drop=True)
    
    train_df, val_df = split_df(df)
    train_df = up_sample(train_df)
    train_df.reset_index(drop=True, inplace=True)
    val_df.reset_index(drop=True, inplace=True)
    
    
    return train_df, val_df

In [17]:
train_data, val_data= process_train(train_data)

In [18]:
train_data

,id,text,label,dataset_type
0,28996,Ơ ... ! Sáng nào tôi cũng uống cafe đen tư...,0,train
1,33230,"Côn đồ, lấy nhầm chồng ko chỉ hại mình mà liên...",1,dev
2,38433,"đó không phải là một dân tộc “anh hùng”, mà l...",0,train
3,34380,"Từ nay hùng hung hăng hùng hổ bớt lại nha, chứ...",1,dev
4,4603,Chỉ sợ muộn quá chị ơi em đang thiếu nợ 10 tri...,0,train
...,...,...,...,...
30379,6469,Bọn bò thì vẫn bảo nhau rằng: tự hào vì ông ch...,1,train
30380,3292,Vào đó từ h oke chưa ??,0,train
30381,25261,Thì GATO với Hoài Linh và các nghệ sĩ hải ngoạ...,0,test
30382,33701,"Gojek mới huy động được 1 tỉ usd, bây h tới lư...",0,dev


In [19]:
train_data.describe()

,id,label
count,30384.000000,30384.000000
mean,22410.263099,0.500000
std,14355.976338,0.500008
min,1.000000,0.000000
25%,8325.750000,0.000000
50%,22874.500000,0.500000
75%,34813.000000,1.000000
max,46006.000000,1.000000


In [20]:
train_data.label.value_counts()

label
0    15192
1    15192
Name: count, dtype: int64

In [21]:
val_data.describe()

,id,label
count,4601.000000,4601.000000
mean,24363.425560,0.163877
std,12492.293956,0.370205
min,13.000000,0.000000
25%,14105.000000,0.000000
50%,24065.000000,0.000000
75%,34452.000000,0.000000
max,46005.000000,1.000000


In [22]:
val_data.label.value_counts()

label
0    3847
1     754
Name: count, dtype: int64

In [23]:
test_data.describe()

,id,label
count,23003.000000,23003.000000
mean,21564.629483,0.193497
std,13701.699345,0.395047
min,0.000000,0.000000
25%,10304.000000,0.000000
50%,18725.000000,0.000000
75%,32791.500000,0.000000
max,44132.000000,1.000000


In [24]:
test_data.label.value_counts()

label
0    18552
1     4451
Name: count, dtype: int64

# **Dataset, Dataloader**

In [25]:
import string

In [26]:
def preprocess_text(text):
    # to lower case
    text = text.lower()
    # remove links
    text = re.sub('https:\/\/\S+', '', text) 
    # remove punctuation
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text) 
    # remove next line     
    text = re.sub(r'[^ \w\.]', '', text) 
    # remove words containing numbers
    text = re.sub('\w*\d\w*', '', text)
    return text

<>:5: SyntaxWarning: invalid escape sequence '\/'
<>:11: SyntaxWarning: invalid escape sequence '\w'
<>:5: SyntaxWarning: invalid escape sequence '\/'
<>:11: SyntaxWarning: invalid escape sequence '\w'
/tmp/ipykernel_2443495/3853289160.py:5: SyntaxWarning: invalid escape sequence '\/'
  text = re.sub('https:\/\/\S+', '', text)
/tmp/ipykernel_2443495/3853289160.py:11: SyntaxWarning: invalid escape sequence '\w'
  text = re.sub('\w*\d\w*', '', text)


### **Data and Model**

In [27]:
tokenizer = AutoTokenizer.from_pretrained("bstrai/multilingual-toxic-xlm-roberta")
model = AutoModelForSequenceClassification.from_pretrained("bstrai/multilingual-toxic-xlm-roberta")

In [28]:
BATCH_SIZE = 16
MAX_LEN = 300

In [29]:
class CustomDataset(Dataset):
    
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text.apply(preprocess_text)
        self.targets = torch.Tensor(list(self.data.label)).reshape(-1, 1)
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            # padding= "max_length",
            pad_to_max_length=True,

            return_token_type_ids=True,
            return_tensors = 'pt',
        )
        ids = inputs['input_ids'].squeeze()
        mask = inputs['attention_mask'].squeeze()
        token_type_ids = inputs["token_type_ids"].squeeze()

        
        return {
            # 'ids': torch.tensor(ids, dtype=torch.long),
            # 'mask': torch.tensor(mask, dtype=torch.long),
            # 'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            # 'targets': torch.tensor(self.targets[index], dtype=torch.float)
            'ids': ids.clone().detach().long(),
            'mask': mask.clone().detach().long(),
            'token_type_ids': token_type_ids.clone().detach().long(),
            'targets': self.targets[index].clone().detach().long(),
            
        }
        

In [30]:
MAX_LEN = args['max_length']
BATCH_SIZE = args['batch_size']
train_dataset = CustomDataset(dataframe= train_data, tokenizer= tokenizer, max_len= MAX_LEN)

In [31]:
for sample in train_dataset:
    print(sample)
    break

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


{'ids': tensor([     0,      6,  12435,   6079,     24,     31,   2259,  16180,     75,
         23366,  43185,  58095,   4797,   3713,   1337, 150421,   2933,   7630,
         75515,     14,      2,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1, 

/work/hpc/miniconda3/envs/hieu-env/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:2688: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [32]:
val_dataset = CustomDataset(dataframe= val_data, tokenizer= tokenizer, max_len= MAX_LEN)
test_dataset = CustomDataset(dataframe= test_data, tokenizer= tokenizer, max_len= MAX_LEN)

In [33]:
train_loader = DataLoader(train_dataset, batch_size= BATCH_SIZE, shuffle= True, num_workers= 4)
val_loader = DataLoader(val_dataset, batch_size= BATCH_SIZE, shuffle= False, num_workers= 4)
test_loader = DataLoader(test_dataset, batch_size= BATCH_SIZE, shuffle= False, num_workers= 4)

In [34]:
train_loader.batch_size

8

In [35]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [36]:
input_ids = next(iter(train_loader))['ids']
print(input_ids.shape)


torch.Size([8, 300])


# **Model**

### **Bert**

In [37]:
in_features = model.classifier
in_features

XLMRobertaClassificationHead(
  (dense): Linear(in_features=768, out_features=768, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (out_proj): Linear(in_features=768, out_features=16, bias=True)
)

In [38]:
print(model)

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768,

# **Model**

In [39]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model.

class Toxic_RoBERTa(torch.nn.Module):
    def __init__(self, mode):
        super(Toxic_RoBERTa, self).__init__()
        self.back_bone = AutoModelForSequenceClassification.from_pretrained("bstrai/multilingual-toxic-xlm-roberta")
        if mode == 'transfer':
            for param in self.back_bone.parameters():
                param.requires_grad = False
        else:
            for param in self.back_bone.parameters():
                param.requires_grad = True
        self.fc = torch.nn.Linear(16, 1)

    def forward(self, ids, mask, token_type_ids):
        output_1 = self.back_bone(ids, attention_mask = mask, token_type_ids= token_type_ids, return_dict=False)[0]
        output = self.fc(output_1)
        return output
    
    



In [40]:
def evaluate(model, data_loader: DataLoader):
    model.to(device)
    MAX_LEN = 300
    BATCH_SIZE = 16
    
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(data_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            
            targets = data['targets'].to(device, dtype = torch.float)
            
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.clone().cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).clone().cpu().detach().numpy().tolist())
#                 
            
    outputs = np.array(fin_outputs) >= 0.5
    accuracy = metrics.accuracy_score(fin_targets, outputs)
    f1_score = metrics.f1_score(fin_targets, outputs, average='binary')
    print(f"Accuracy Score = {accuracy}")
    print(f"F1 Score = {f1_score}")
    
    return fin_outputs, fin_targets, accuracy, f1_score

def predict(model, text):
    model.to(device)
    model.eval()
    inputs = tokenizer.encode_plus(text, 
                        add_special_tokens=True, 
                        return_token_type_ids= True,
                        max_length=MAX_LEN, 
                        padding="max_length", 
                        return_tensors='pt') 
    
    ids = inputs['input_ids'].to(device)
    mask = inputs['attention_mask'].to(device)
    token_type_ids = inputs["token_type_ids"].to(device)
    # print(ids.shape)

    # ids = ids.to(device)
    # mask = mask.to(device)
    # token_type_ids = token_type_ids.to(device)
            
    outputs = model(ids, mask, token_type_ids)
    return torch.sigmoid(outputs).clone().cpu().detach().numpy().tolist()

In [41]:
def get_model(mode= 'fine_tune', weight_file= None):
    
    model = Toxic_RoBERTa(mode= mode)
    # print('a')
    
    if weight_file:
        print("load weight from", weight_file)
        model.load_state_dict(torch.load(weight_file, map_location= device))
    return model

# **Train**

In [42]:
def get_Loss_func():
    return torch.nn.BCEWithLogitsLoss()

def model_step(batch, model, loss_func, device):
    ids = batch['ids'].to(device, dtype = torch.long)
    mask = batch['mask'].to(device, dtype = torch.long)
    token_type_ids = batch['token_type_ids'].to(device, dtype = torch.long)
    targets = batch['targets'].to(device, dtype = torch.float)

    outputs = model(ids, mask, token_type_ids)
    loss = loss_func(outputs, targets)
    preds = outputs >= 0.5

    return loss, preds, targets

def optimizer_step(optimizer, loss):
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

def evaluation(model, loader, loss_func, epoch, device, ):
    model.eval()
    total, total_loss, total_correct = 0, 0, 0
    preds_labels = []
    y_labels = []
    pbar = tqdm(loader)
    cnt = 0
    for batch in pbar:
        loss, preds, y = model_step(batch, model, loss_func, device)

        total += len(y)
        total_loss += loss.item()
        total_correct += torch.sum(preds == y).item()
        pbar.set_description(f"epoch = {epoch}, val/loss = {total_loss/total:.3f}, val/acc = {total_correct/total:.3f}")
        preds_labels.extend(preds.clone().cpu().detach().numpy().tolist())
        y_labels.extend(y.clone().cpu().detach().numpy().tolist())
    
    accuracy = accuracy_score(y_labels, preds_labels)
    precision = precision_score(y_labels, preds_labels)
    recall = recall_score(y_labels, preds_labels)
    f1 = f1_score(y_labels, preds_labels)
    
    wandb.log({"val_loss": total_loss/total, "val_acc": total_correct/total, "val_precision": precision, "val_recall": recall, "val_f1": f1})
    

    return (total_loss / total), (total_correct/total)    #loss, accuracy

def train_epoch(model, train_loader, loss_func, optimizer, epoch, device):
    model.train()
    pbar = tqdm(train_loader)
    total, total_loss, total_correct = 0, 0, 0
    cnt = 0
    for batch in pbar:
        loss, preds, y = model_step(batch, model, loss_func, device)
        optimizer_step(optimizer, loss)

        total += len(y)
        total_loss += loss.item()
        total_correct += torch.sum(preds == y).item()
        pbar.set_description(f"epoch = {epoch}, train/loss = {total_loss/total:.3f}, train/acc = {total_correct/total:.3f} ")
        
        if cnt % 40 == 0:
            wandb.log({"train_loss": total_loss/total, "train_acc": total_correct/total})
        cnt += 1
    
    return (total_loss / total), (total_correct/total)    #loss, accuracy


def plot_training_history(train_loss_history, train_acc_history, val_loss_history, val_acc_history):
    plt.figure(figsize=(8, 8))
    plt.subplot(2, 1, 1)
    plt.plot(train_acc_history, label='Training Accuracy')
    plt.plot(val_acc_history, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.ylabel('Accuracy')
    plt.title('Training and Validation Accuracy')

    plt.subplot(2, 1, 2)
    plt.plot(train_loss_history, label='Training Loss')
    plt.plot(val_loss_history, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.ylabel('BCElogits Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('epoch')
    plt.show()

def train(model, train_loader, val_loader, lr=1e-3, max_epochs=1, device = "cuda", plot_res = True, output_file=None):
    train_loss_history = []
    train_acc_history = []
    val_loss_history = []
    val_acc_history = []
    loss_func = get_Loss_func()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    best_score = 0
    for epoch in range(max_epochs):
        ## train loop
        train_loss, train_acc = train_epoch(model, train_loader, loss_func, optimizer, epoch, device)
        train_loss_history.append(train_loss)
        train_acc_history.append(train_acc)
        ## val loop
        val_loss, val_acc = evaluation(model, val_loader, loss_func, epoch, device)
        val_loss_history.append(val_loss)
        val_acc_history.append(val_acc)
        # wandb.log({"train_loss": train_loss, "train_acc": train_acc, "val_loss": val_loss, "val_acc": val_acc})
        
        if output_file and val_acc > best_score:
            best_score = val_acc
            save_weight(model, output_file)
            
    if(plot_res):
        plot_training_history(train_loss_history, train_acc_history, val_loss_history, val_acc_history)
    return (train_loss_history, train_acc_history, val_loss_history, val_acc_history)

def save_weight(model, output_file):
    print("save model to", output_file)
    torch.save(model.state_dict(), output_file[:-4] + "_best" + ".pth")
    
def main(mode="fine-tune", weight_file=None, output_file=None, epochs= None, device= 'cuda'):

    if mode == "fine-tune" and weight_file:
        model = get_model(mode=mode, weight_file=weight_file)
    else:
        model = get_model(mode=mode)
    
    model.to(device)
    wandb.watch(model, log="all")
    lr = 1e-3 if mode == "transfer" else 1e-5
    max_epochs = epochs 
    train(model, train_loader, val_loader, device=device, max_epochs=max_epochs, lr=lr, output_file=output_file)

    if output_file:
        # 
        save_weight(model, output_file)

In [43]:
output_fine_tune_link = 'ckpt/model-fine-tune1.2.pth'
output_transfer_link = 'ckpt/model-transfer.pth'

In [45]:
# output_folder_link = 'ckpt'

# main(mode="transfer", output_file= output_transfer_link, device= device, epochs= 1)
main(mode="fine-tune", weight_file= "ckpt/model-fine-tune1.2_best.pth", output_file= output_fine_tune_link, device= device, epochs= args['fine_tune_epochs'])


load weight from ckpt/model-fine-tune1.2_best.pth


KeyboardInterrupt: 


## Validating the Model

As defined above to get a measure of our models performance we are using the following metrics.
- Accuracy Score
- F1 Micro
- F1 Macro

In [46]:
# model = get_model(weight_file= output_fine_tune_link)
model = get_model(weight_file= "ckpt/model-fine-tune1.2_best.pth")

model.to(device)
model.eval()

load weight from ckpt/model-fine-tune1.2_best.pth


Toxic_RoBERTa(
  (back_bone): XLMRobertaForSequenceClassification(
    (roberta): XLMRobertaModel(
      (embeddings): XLMRobertaEmbeddings(
        (word_embeddings): Embedding(250002, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): XLMRobertaEncoder(
        (layer): ModuleList(
          (0-11): 12 x XLMRobertaLayer(
            (attention): XLMRobertaAttention(
              (self): XLMRobertaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): XLMRobertaSelfOutp

In [47]:
final_output, final_target, acc, f1 = evaluate(model, test_loader)

/work/hpc/miniconda3/envs/hieu-env/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:2688: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/work/hpc/miniconda3/envs/hieu-env/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:2688: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave

Accuracy Score = 0.8471068991001174
F1 Score = 0.5638099962793005


In [49]:
output = (np.array(final_output) >= 0.5).astype(int)
output

array([[0],
       [0],
       [0],
       ...,
       [1],
       [1],
       [0]])

In [50]:
final_target = np.array(final_target).astype(int)
final_target

array([[0],
       [0],
       [0],
       ...,
       [1],
       [1],
       [0]])

In [55]:
del recall_score, f1_score, precision_score

In [60]:
accuracy = accuracy_score(final_target, output)
recall = recall_score(final_target, output)
precision = precision_score(final_target, output)
f1 = f1_score(final_target, output)
print(f'accuracy: ', accuracy)
print(f'recall: ', recall)
print(f'precision: ', precision)
print(f'f1: ', f1)

accuracy:  0.8471068991001174
recall:  0.510671759155246
precision:  0.6292912513842747
f1:  0.5638099962793005


In [61]:
test_data['predict_label'] = output
test_data['predict_label'] = test_data['predict_label'].astype(int)

In [62]:
mislabel = test_data[test_data.label != test_data.predict_label]

In [63]:
mislabel

,id,text,label,dataset_type,predict_label
7,32375,Tôi không hiểu sao lại để vậy.,0,train,1
22,5553,Không biết tốt xấu! Chất phát như nào nhưng ng...,1,train,0
29,7102,con này thực sự nó giống như con thú nhô...,1,train,0
36,43278,chuyện đời cộng sản lưu vong (7) ***** 21) mườ...,1,train,0
48,6289,Cái quần in hệt cái ý thức,1,train,0
...,...,...,...,...,...
22965,42839,BỘ CÔNG AN TUYỂN HƠN 16.000 LÍNH CSCĐ Năm 2024...,1,train,0
22980,40625,vậy điều 331 bộ luật hình sự 2015 có nội dung...,1,train,0
22981,37954,khu ký túc xá nằm gần với bệnh viện đa khoa t...,0,train,1
22983,3196,"Tú Trinh ko chạy ăn cứt, ăn đb à 🙄",1,train,0


In [64]:
mislabel.to_csv('Iterations/Iter1/1.2/mislabel.csv', index= False)